## Targets is S&P500 returns, input is either from FRED St. Louis or Yahoo finance

In [6]:
import numpy as np
import pandas as pd

#### Read in the targets. Periods of downturn are periods where consecutive monthly returns <-15%

In [7]:
df = pd.read_csv('targets.csv')
df = df[['Date','Open','Close','monthly_return','period of downturn','Recessions']]
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Open,Close,monthly_return,period of downturn,Recessions
0,1950-01-01,16.660000,17.049999,0.023409,0,0
1,1950-02-01,17.049999,17.219999,0.009971,0,0
2,1950-03-01,17.240000,17.290001,0.002900,0,0
3,1950-04-01,17.340000,18.070000,0.042099,0,0
4,1950-05-01,18.219999,18.780001,0.030736,0,0


#### Read in Industrial Production from FRED STL, lagging by 2 months for results to come out

In [8]:
indpro = pd.read_csv('INDPRO.csv')
indpro.columns = ['Date','INDPRO']
indpro['Date'] = pd.to_datetime(indpro['Date'])

df = df.merge(indpro, how='left', on='Date')
df['INDPRO'] = df['INDPRO'].shift(2)

#### Read in daily yield curve and match with the monthly stock result using fuzzy logic (yc data was available for weekdays only)

In [9]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [10]:
yield_curve = pd.read_csv('T10Y2Y.csv')
yield_curve.columns = ['Date','YIELD_CURVE']
yield_curve['Date'] = pd.to_datetime(yield_curve['Date'])
yield_curve['YIELD_CURVE'] = np.where(yield_curve['YIELD_CURVE']=='.',None,yield_curve['YIELD_CURVE'])
yield_curve['YIELD_CURVE'].fillna(method='ffill', inplace=True)

df['YIELD_CURVE'] = df['Date'].map(lambda x:nearest(yield_curve['Date'],x))    



In [11]:
df.rename(columns={'YIELD_CURVE':'yc_date'},inplace=True)

#### Accounting for the yc data starting in 1976

In [12]:
df1 = df.merge(yield_curve,how='left',left_on='yc_date',right_on='Date')
df1['YIELD_CURVE'] = np.where(df1['Date_y']==pd.to_datetime('1976-06-01'),None,df1['YIELD_CURVE'])

#### Read in unemployment and lag by 2 months for announcement time. Also checkpoint because fuzzy matching took some time

In [13]:
df1.drop(columns=['Date_y'],inplace=True)
df1.rename(columns={'Date_x':'Date'},inplace=True)

unemploy = pd.read_csv('UNRATE.csv')
unemploy.columns = ['Date','UNEMPLOYMENT']
unemploy['Date'] = pd.to_datetime(unemploy['Date'])

df1 = df1.merge(unemploy, how='left', on='Date')
df1['UNEMPLOYMENT'] = df1['UNEMPLOYMENT'].shift(2)

df1.to_csv('zzz_df_checkpoint.csv',index=False)

#### Read in checkpoint_df

In [14]:
df1 = pd.read_csv('zzz_df_checkpoint.csv')
df1['Date'] = pd.to_datetime(df1['Date'])

#### Read in individual stock data, get difference from 2-month high and merge

In [15]:
tickers = ['BGI','KO','KR','MAR','PPG','SHW']
for tick in tickers:
    new_df = pd.read_csv(tick+'.csv')
    new_df.columns = ['Date','Open','High','Low',tick+'_Close','Adj Close','Volume']
    new_df = new_df[['Date',tick+'_Close']]
    new_df['Date'] = pd.to_datetime(new_df['Date'])
    new_df[tick+'_2_mth_high'] = new_df['Date'].map(lambda x:max(new_df[(new_df['Date']<=pd.to_datetime(x))&(new_df['Date']>=pd.to_datetime(x)-pd.Timedelta(35,'d'))][tick+'_Close']))
    new_df[tick+'_diff_from_2_month_high'] = (new_df[tick+'_Close'] - new_df[tick+'_2_mth_high'])/new_df[tick+'_2_mth_high']
    new_df = new_df[['Date',tick+'_diff_from_2_month_high']]
    
    df1 = df1.merge(new_df,how='left',on='Date')
    df1[tick+'_diff_from_2_month_high'] = df1[tick+'_diff_from_2_month_high'].shift(1)

#### Create difference from 6 month high(low) for unemployment and ind pro

In [16]:
df1 = df1[df1['Date']!=pd.to_datetime('1950-01-01')]

df1['unemployment_6_mth_low'] = df1['Date'].map(lambda x:min(df1[(df1['Date']<=pd.to_datetime(x))&(df1['Date']>=pd.to_datetime(x)-pd.Timedelta(180,'d'))]['UNEMPLOYMENT']))
df1['unemployment_diff_from_6_month_low'] = df1['UNEMPLOYMENT'] - df1['unemployment_6_mth_low']

df1['INDPRO_6_mth_high'] = df1['Date'].map(lambda x:max(df1[(df1['Date']<=pd.to_datetime(x))&(df1['Date']>=pd.to_datetime(x)-pd.Timedelta(180,'d'))]['INDPRO']))
df1['INDPRO_diff_from_6_month_high'] = df1['INDPRO_6_mth_high'] - df1['INDPRO']


df1.drop(columns=['unemployment_6_mth_low','INDPRO_6_mth_high'],inplace=True)
df1.head()

,Date,Open,Close,monthly_return,period of downturn,Recessions,INDPRO,yc_date,YIELD_CURVE,UNEMPLOYMENT,BGI_diff_from_2_month_high,KO_diff_from_2_month_high,KR_diff_from_2_month_high,MAR_diff_from_2_month_high,PPG_diff_from_2_month_high,SHW_diff_from_2_month_high,unemployment_diff_from_6_month_low,INDPRO_diff_from_6_month_high
1,1950-02-01,17.049999,17.219999,0.009971,0,0,NaN,1976-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1950-03-01,17.240000,17.290001,0.002900,0,0,14.4279,1976-06-01,NaN,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1950-04-01,17.340000,18.070000,0.042099,0,0,14.4833,1976-06-01,NaN,6.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-05-01,18.219999,18.780001,0.030736,0,0,14.9541,1976-06-01,NaN,6.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1950-06-01,18.770000,17.690001,-0.057539,0,0,15.4526,1976-06-01,NaN,5.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df1.to_csv('reg_ready.csv',index=False)